In [15]:
import pandas as pd
import numpy as np
import os
from scipy.stats import linregress, ttest_ind

# Function to compute linear trend (slope) per decade and p-value
def compute_decadal_slope(x, y):
    """
    Performs linear regression on time series data and scales slope to represent change per decade.

    Parameters:
        x (array-like): Time (years)
        y (array-like): Variable values over time

    Returns:
        slope (float): Trend per decade
        p_value (float): P-value of the regression
    """
    res = linregress(x, y)
    return res.slope * 10, res.pvalue  # Convert from per year to per decade

# Function to analyze climate statistics for a single city
def analyze_city(city, path):
    """
    Analyzes climate trends and scenario differences for one city.

    Parameters:
        city (str): City name
        path (str): File path to the city's climate data CSV

    Returns:
        dict: Summary of climate indicators for temperature and precipitation
    """
    df = pd.read_csv(path)
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year

    # Focus analysis on 2020–2100 to avoid transition issues from historical to scenario data
    df_trend = df[df['year'].between(2020, 2100)]
    df_early = df[df['year'].between(2020, 2040)]
    df_end_century = df[df['year'].between(2080, 2100)]

    # Initialize results
    ssp245_trend = ssp585_trend = None
    ssp245_prec_trend = ssp585_prec_trend = None
    delta_T_245 = delta_T_585 = None
    delta_p_245 = delta_p_585 = None
    p_val_T = p_val_P = None

    # Calculate temperature trends per decade for each scenario
    for scenario in ['ssp245', 'ssp585']:
        temp_data = df_trend[df_trend['scenario'] == scenario]
        yearly_temp = temp_data.groupby('year')['temperature_C'].mean().reset_index()
        slope, _ = compute_decadal_slope(yearly_temp['year'], yearly_temp['temperature_C'])
        if scenario == 'ssp245':
            ssp245_trend = slope
        else:
            ssp585_trend = slope

    # Calculate precipitation trends per decade for each scenario
    for scenario in ['ssp245', 'ssp585']:
        prec_data = df_trend[df_trend['scenario'] == scenario]
        yearly_prec = prec_data.groupby('year')['precipitation_mm_per_day'].mean().reset_index()
        slope, _ = compute_decadal_slope(yearly_prec['year'], yearly_prec['precipitation_mm_per_day'])
        if scenario == 'ssp245':
            ssp245_prec_trend = slope
        else:
            ssp585_prec_trend = slope

    # Calculate temperature deltas (2080–2100 vs 2020–2040)
    temp_early_245 = df_early[df_early['scenario'] == 'ssp245']['temperature_C']
    temp_end_245 = df_end_century[df_end_century['scenario'] == 'ssp245']['temperature_C']
    temp_early_585 = df_early[df_early['scenario'] == 'ssp585']['temperature_C']
    temp_end_585 = df_end_century[df_end_century['scenario'] == 'ssp585']['temperature_C']

    if not temp_early_245.empty and not temp_end_245.empty:
        delta_T_245 = temp_end_245.mean() - temp_early_245.mean()
    if not temp_early_585.empty and not temp_end_585.empty:
        delta_T_585 = temp_end_585.mean() - temp_early_585.mean()

    # Perform t-test on end-century temperatures between SSP scenarios
    temp_245 = df_end_century[df_end_century['scenario'] == 'ssp245']['temperature_C']
    temp_585 = df_end_century[df_end_century['scenario'] == 'ssp585']['temperature_C']
    if not temp_245.empty and not temp_585.empty:
        _, p_val_T = ttest_ind(temp_585, temp_245, equal_var=False)

    # Calculate precipitation deltas (2080–2100 vs 2020–2040)
    prec_early_245 = df_early[df_early['scenario'] == 'ssp245']['precipitation_mm_per_day']
    prec_end_245 = df_end_century[df_end_century['scenario'] == 'ssp245']['precipitation_mm_per_day']
    prec_early_585 = df_early[df_early['scenario'] == 'ssp585']['precipitation_mm_per_day']
    prec_end_585 = df_end_century[df_end_century['scenario'] == 'ssp585']['precipitation_mm_per_day']

    if not prec_early_245.empty and not prec_end_245.empty:
        delta_p_245 = 100 * (prec_end_245.mean() - prec_early_245.mean()) / prec_early_245.mean()
    if not prec_early_585.empty and not prec_end_585.empty:
        delta_p_585 = 100 * (prec_end_585.mean() - prec_early_585.mean()) / prec_early_585.mean()

    # Perform t-test on end-century precipitation between SSP scenarios
    if not prec_end_245.empty and not prec_end_585.empty:
        _, p_val_P = ttest_ind(prec_end_585, prec_end_245, equal_var=False)

    # Compile results
    return {
        "Location": city,
        "SSP2-4.5 Trend (°C/decade)": round(ssp245_trend, 2),
        "SSP5-8.5 Trend (°C/decade)": round(ssp585_trend, 2),
        "ΔT SSP2-4.5 (2020–2100)": f"{delta_T_245:+.1f}°C" if delta_T_245 is not None else None,
        "ΔT SSP5-8.5 (2020–2100)": f"{delta_T_585:+.1f}°C" if delta_T_585 is not None else None,
        "p-value (Temp)": f"{p_val_T:.2g}" if p_val_T is not None else None,
        "SSP2-4.5 Trend (mm/day/decade)": round(ssp245_prec_trend, 3),
        "SSP5-8.5 Trend (mm/day/decade)": round(ssp585_prec_trend, 3),
        "SSP2-4.5 ΔP (%)": f"{delta_p_245:+.0f}%" if delta_p_245 is not None else None,
        "SSP5-8.5 ΔP (%)": f"{delta_p_585:+.0f}%" if delta_p_585 is not None else None,
        "p-value (Precip)": f"{p_val_P:.2g}" if p_val_P is not None else None
    }

# Main execution function to process all cities and export summary tables
def main():
    input_dir = "Cmip6_extracts"
    output_dir = "Summary_Tables"
    os.makedirs(output_dir, exist_ok=True)

    files = {
        "Cairo": os.path.join(input_dir, "Cairo_data.csv"),
        "Helsinki": os.path.join(input_dir, "Helsinki_data.csv"),
        "New Delhi": os.path.join(input_dir, "New_Delhi_data.csv"),
    }

    # Run analysis for each city and collect results
    summary_stats = [analyze_city(city, path) for city, path in files.items()]
    df_summary = pd.DataFrame(summary_stats)

    # Temperature summary
    df_temp = df_summary[[
        "Location",
        "SSP2-4.5 Trend (°C/decade)",
        "SSP5-8.5 Trend (°C/decade)",
        "ΔT SSP2-4.5 (2020–2100)",
        "ΔT SSP5-8.5 (2020–2100)",
        "p-value (Temp)"
    ]]

    # Precipitation summary
    df_precip = df_summary[[
        "Location",
        "SSP2-4.5 Trend (mm/day/decade)",
        "SSP5-8.5 Trend (mm/day/decade)",
        "SSP2-4.5 ΔP (%)",
        "SSP5-8.5 ΔP (%)",
        "p-value (Precip)"
    ]]

    # Save to output directory
    df_temp.to_csv(os.path.join(output_dir, "Temperature_Trends_2020_2100.csv"), index=False)
    df_precip.to_csv(os.path.join(output_dir, "Precipitation_Changes_2020_2100.csv"), index=False)

    print("Results saved to CSV files in Summary_Tables")

# ------------------------------------------------------------------------------
# Entry point
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    main()


Results saved to CSV files in Summary_Tables
